In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [22]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np 
import time
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
import pandas as pd

#Import library tree from sklearn
from sklearn import tree , metrics

# Import datasets, classifiers and performance metrics
from sklearn.model_selection import train_test_split

### Chargement des données

In [23]:
train1 = pd.read_csv("/kaggle/input/smemi309-final-evaluation-challenge-2021/train1slice.csv")
train2 = pd.read_csv("/kaggle/input/smemi309-final-evaluation-challenge-2021/train2slices.csv")
train3 = pd.read_csv("/kaggle/input/smemi309-final-evaluation-challenge-2021/train3slices.csv")
train4 = pd.read_csv("/kaggle/input/smemi309-final-evaluation-challenge-2021/train4slices.csv")
train5 = pd.read_csv("/kaggle/input/smemi309-final-evaluation-challenge-2021/train5slices.csv")


test1 = pd.read_csv("/kaggle/input/smemi309-final-evaluation-challenge-2021/test1slice.csv")
test2 = pd.read_csv("/kaggle/input/smemi309-final-evaluation-challenge-2021/test2slices.csv")
test3 = pd.read_csv("/kaggle/input/smemi309-final-evaluation-challenge-2021/test3slices.csv")
test4 = pd.read_csv("/kaggle/input/smemi309-final-evaluation-challenge-2021/test4slices.csv")
test5 = pd.read_csv("/kaggle/input/smemi309-final-evaluation-challenge-2021/test5slices.csv")

### selection models and prediction

In [24]:
## Définission de l'ensemble de test et de l'ensemble d'apprentissage
X_train5 = train5.loc[:,train5.columns != 'label']

y_train5 = train5.label


### Data Spliting 

In [ ]:
# Split data into 70% train and 30% test subsets
#from sklearn.model_selection import train_test_split

#X = train5.drop(['label'],axis=1).values   # independant features
#y = train5['label'].values                # dependant variable

# Choose your test size to split between training and testing sets:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

## Utilisation de Gridsearch

In [25]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import validation_curve
from sklearn.model_selection import learning_curve

from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier

### Random Forest

In [26]:
## Random Forest

param_grid = {'n_estimators': np.arange(10,120,10),'criterion':["gini","entropy"],
             'max_features':["auto","sqrt","log2"]}
grid1 = GridSearchCV(RandomForestClassifier(),param_grid,cv=5)
grid1.fit(X_train5,y_train5)

In [29]:
print(grid1.best_score_,grid1.best_params_,grid1.best_estimator_)

In [30]:
# Selection of the best configuration
model1 = grid1.best_estimator_

In [ ]:
# ######## A REVOIR #######################################

# ##Learning curve 

# N, train_score,val_score = learning_curve(model,X_train1,y_train1,train_sizes = np.linspace(0.1,1.0,10),cv=5)
# print(N)  # N = nombre de lots
# plt.plot(N,train_score.mean(axis=1),'b',label='train')
# plt.plot(N,val_score.mean(axis=1),'--r',label='validation')
# plt.xlabel('train_sizes')

### Adaboost

In [31]:
k = np.arange(1,21,1)        ### Max_depth values
mytree = [DecisionTreeClassifier(max_depth=i) for  i in k]
param_grid = {'base_estimator': mytree,'n_estimators':np.arange(10,500,20)}
grid2 = GridSearchCV(AdaBoostClassifier(),param_grid,cv=5)
grid2.fit(X_train5,y_train5)

In [ ]:
# grid.best_estimator_


In [ ]:
print(grid2.best_score_,grid2.best_params_,grid2.best_estimator_)

In [ ]:
# Selection of the best configuration
model2 = grid2.best_estimator_

In [ ]:
# Predict the value of the digit on the test subset
#predicted =model2.predict(test5)
#output = pd.DataFrame({'label':predicted})
#output['id'] = range(0,len(output))

#columnsTitles=["id","label"]
#output = output.reindex(columns=columnsTitles)
#output.to_csv('my_submission.csv', index=False)

## Bagging

In [ ]:
k = np.arange(1,21,2)        ### Max_depth values
mytree = [DecisionTreeClassifier(max_depth=i) for  i in k]

In [ ]:
param_grid = {'base_estimator': mytree,'n_estimators':np.arange(10,120,20)}
grid3 = GridSearchCV(BaggingClassifier(),param_grid,cv=5)
grid3.fit(X_train5,y_train5)

In [ ]:
print(grid3.best_score_,grid3.best_params_,grid3.best_estimator_)

In [ ]:
# Selection of the best configuration
model3 = grid3.best_estimator_


## XGboost

In [ ]:
### Max_depth values
param_grid = {'max_depth':np.arange(1,21,2),'n_estimators':np.arange(10,120,20)}
grid4 = GridSearchCV(GradientBoostingClassifier(),param_grid,cv=5)
grid4.fit(X_train5,y_train5)

In [ ]:
print(grid4.best_score_,grid4.best_params_,grid4.best_estimator_)

In [ ]:
# Selection of the best configuration
model4 = grid4.best_estimator_

## Best Model

In [ ]:

# according to the highest accuracy this function return automaticaly the best model

EnsModels = {'model1':grid1.best_score_, 'model2':grid2.best_score_, 'model3': grid3.best_score_,'model4': grid4.best_score_}

def keywithmaxval(EnsModels):
         #a) create a list of the dict's keys and values 
         #b) return the key with the max value 
     v=list(EnsModels.values())
     k=list(EnsModels.keys())
     return k[v.index(max(v))]

In [ ]:
#Best Model
BestModel=keywithmaxval(EnsModels)

## Prediction

In [ ]:
# Predict the value of the digit on the test subset
predicted =BestModel.predict(test5)

In [ ]:
output = pd.DataFrame({'label':predicted})
output['id'] = range(0,len(output))

columnsTitles=["id","label"]
output = output.reindex(columns=columnsTitles)
output.to_csv('my_submission.csv', index=False)

In [ ]:
#Converter in dataframe
#dfpredicted =pd.DataFrame({'label': predicted})

In [ ]:
#dfpredicted['id'] = dfpredicted.index
#dfpredicted.to_csv('my_submission.csv', index=False)
#print("Your submission was successfully saved!")